In [ ]:
import requests
from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, urlparse, urldefrag
import time
import re
import wikipediaapi
from fake_useragent import UserAgent
import wikipediaapi
import urllib.parse
import time
import re 


In [ ]:
ua = UserAgent()
USER_AGENT = ua.random

REQUEST_TIMEOUT = 10  
CRAWL_DELAY = 1 
SESSION = requests.Session()
SESSION.headers.update({"User-Agent": ua.random})
# MAX_PAGES_PER_DOMAIN = 100 

In [ ]:

def get_page_html_from_api(page_title, lang):
    
    api_url = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "parse",
        "page": page_title,
        "prop": "text",       
        "format": "json",
        "disabletoc": True,    
        "disableeditsection": True
    }
    try:
        response = SESSION.get(url=api_url, params=params)
        response.raise_for_status()
        data = response.json()
        if "parse" in data and "text" in data["parse"] and "*" in data["parse"]["text"]:
            return data["parse"]["text"]["*"]
        else:
            print(f"Could not find parsed text for '{page_title}' on {lang}.wikipedia.org")
            print(f"API Response: {data}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error fetching HTML for '{page_title}': {e}")
        return None


In [ ]:
def get_wiki_lang_and_title_from_url(url):
    try:
        parsed_url = urllib.parse.urlparse(url)
        hostname = parsed_url.hostname # 'vi'
        path_parts = parsed_url.path.strip('/').split('/') # ['wiki', 'title']

        lang_match = re.match(r"([a-z]{2,3}(?:-[a-z]+)?)", hostname)
        lang = lang_match.group(1)

        if len(path_parts) >= 2 and path_parts[0].lower() == 'wiki':
            page_title_encoded = path_parts[1]
            page_title = urllib.parse.unquote(page_title_encoded).replace('_', ' ')
            return lang, page_title
        else:
            print(f"Error: URL path does not match expected '/wiki/Page_Title' structure: {parsed_url.path}")
            return None, None

    except Exception as e:
        print(f"Error parsing {url}': {e}")
        return None, None

In [107]:
def process_monument_links_from_table(seed_url):
    lang, seed_page_title = get_wiki_lang_and_title_from_url(seed_url)


    
    monument_column_header="Di tích"

    # Get the HTML of the seed page
    page_html_content = get_page_html_from_api(seed_page_title, lang)


    soup = BeautifulSoup(page_html_content, 'lxml')
    monument_titles = set() 

    # Find tables 
    tables = soup.find_all('table', class_='wikitable')
    if not tables:
        print("No 'wikitable' found on the page.")


    found_monument_column = False
    for table in tables:
        headers = [th.get_text(strip=True) for th in table.find_all('th')]
        monument_col_idx = -1

        for i, header_text in enumerate(headers):
            if monument_column_header.lower() in header_text.lower(): 
                monument_col_idx = i
                found_monument_column = True
                break

        if monument_col_idx != -1:
            print(f"Found '{monument_column_header}' column at index {monument_col_idx} in a table.")
            # Iterate through rows of this table
            for row in table.find_all('tr'):
                cells = row.find_all(['td', 'th']) #
                if len(cells) > monument_col_idx:
                    monument_cell = cells[monument_col_idx]
                    # Find all a tags 
                    for link_tag in monument_cell.find_all('a', href=True):
                        href = link_tag['href']
                        # Check internal Wikipedia link
                        if href.startswith('/wiki/') and ':' not in href[6:]:
                            link_title_encoded = href.split('/wiki/')[-1]
                            # Avoid links to the same page
                            if '#' not in link_title_encoded:
                                link_title = urllib.parse.unquote(link_title_encoded).replace('_', ' ')
                                #filter out red links
                                if 'class' not in link_tag.attrs or 'new' not in link_tag['class']:
                                    monument_titles.add(link_title)

    


    if not monument_titles:
        print(f"No monument links found in the '{monument_column_header}' column.")
        return

    print(f"\nFound {len(monument_titles)} unique potential monument titles from the table column.")


    return monument_titles, lang

# vietnamese_monuments_url = 'https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_Di_t%C3%ADch_qu%E1%BB%91c_gia_Vi%E1%BB%87t_Nam'

# sec_url = 'https://vi.wikipedia.org/wiki/Di_t%C3%ADch_qu%E1%BB%91c_gia_%C4%91%E1%BA%B7c_bi%E1%BB%87t_(Vi%E1%BB%87t_Nam)'
# # The header text for the monument column on the Vietnamese page is "Di tích"
# monument_titles = process_monument_links_from_table(vietnamese_monuments_url)


In [ ]:
exclude_section_titles_lower = [
"xem thêm", "chú thích", "liên kết ngoài", "tham khảo", "đọc thêm",
"gallery", "hình ảnh", "notes", "references", "external links", "see also",
"bibliography", "sources" 
]
def get_clean_text_from_wiki_page(page_title,lang,wiki_api_instance):

    page = wiki_api_instance.page(page_title)

    if not page.exists():
        print(f"  Page '{page_title}' ({lang}) does not exist via wikipediaapi.")
        return ""

    formatted_text_parts = []


    for section in page.sections:
        section_title_cleaned = section.title.strip()
        if section_title_cleaned.lower() not in exclude_section_titles_lower:
            sec_text = section.text.strip()

            if sec_text: 
                formatted_text_parts.append(f"# {section_title_cleaned}\n{sec_text}")

    full_text = "\n\n".join(formatted_text_parts).strip()
    
    return full_text

def fetch_monument_page_texts(
    monument_titles: set,
    lang: str,
    max_pages_to_fetch: int = None,
    user_agent_for_content: str = USER_AGENT,
):
    if not monument_titles:
        print("No monument titles provided to fetch texts for.")
        return []

    print(f"\nFetching text for up to {max_pages_to_fetch if max_pages_to_fetch else len(monument_titles)} monument pages on {lang}.wikipedia.org...")
    
    wiki_api = wikipediaapi.Wikipedia(
        language=lang,
        user_agent=user_agent_for_content,
        extract_format=wikipediaapi.ExtractFormat.WIKI
    )

    all_monument_data = []
    count = 0
    for title in monument_titles:
        if max_pages_to_fetch is not None and count >= max_pages_to_fetch:
            print(f"Reached max pages limit of {max_pages_to_fetch}.")
            break
        
        print(f"  Fetching for: '{title}'...")
        page_text = get_clean_text_from_wiki_page(
            title,
            lang,
            wiki_api,
        )

        if page_text:
            all_monument_data.append({
                "title": title,
                "language": lang,
                "url": f"https://{lang}.wikipedia.org/wiki/{urllib.parse.quote(title.replace(' ', '_'))}",
                "text_content": page_text
            })
        else:
            print(f"  No text extracted for '{title}'.")
        
        count += 1
        time.sleep(0.5)

    return all_monument_data


In [112]:
import json

seed_url ='https://vi.wikipedia.org/wiki/Di_t%C3%ADch_qu%E1%BB%91c_gia_%C4%91%E1%BA%B7c_bi%E1%BB%87t_(Vi%E1%BB%87t_Nam)'
max_pages_to_process_from_list = 620
output_json_file = "monument_database_no_summary_2.json"

monument_titles_set, site_lang = process_monument_links_from_table(seed_url)

if monument_titles_set and site_lang:
    print(f"\n--- Successfully extracted {len(monument_titles_set)} titles")


    print(f"\nStep 2: Fetching text content for up to {max_pages_to_process_from_list or len(monument_titles_set)} monument pages...")
    extracted_data = fetch_monument_page_texts(
        monument_titles=monument_titles_set,
        lang=site_lang,
        max_pages_to_fetch=max_pages_to_process_from_list,
        user_agent_for_content=USER_AGENT
    )


    try:
        with open(output_json_file, 'a', encoding='utf-8') as f:
            json.dump(extracted_data, f, ensure_ascii=False, indent=4)
        print(f"\nSaved detailed data for {len(extracted_data)} monuments to '{output_json_file}'")
    except IOError as e:
        print(f"Error saving data to JSON file: {e}")

else:
    print("\nCould not extract monument titles from the seed URL. Pipeline halted.")



Found 'Di tích' column at index 1 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.
Found 'Di tích' column at index 0 in a table.

Found 132 unique potential monument titles from the table column.

--- Successfully extracted 132 titles

Step 2: Fetching te

In [78]:
wiki_wiki = wikipediaapi.Wikipedia(language='vi', user_agent=ua.random)


In [79]:
page = wiki_wiki.page('Thành Bản Phủ (Điện Biên)')

In [ ]:
words = page.text

In [83]:
words

'Thành Bản Phủ là thành lũy được Hoàng Công Chất xây dựng ở châu Ninh Biên, phủ An Tây làm thủ phủ cho nghĩa quân. Ngày nay Thành Bản Phủ tọa lạc ở xã Noong Hẹt, huyện Điện Biên, cách thành phố Điện Biên Phủ khoảng 9 km.\nThành Bản Phủ được Bộ Văn hóa và Thông tin xếp hạng là di tích lịch sử quốc gia tại Quyết định số 10-VHTT/QĐ ngày 09/02/1981.\n\nTruyền thuyết giặc Phẻ\nĐầu của thế kỷ 18 xuất hiện giặc Phẻ từ phương Bắc tràn xuống vùng Mường Thanh, cướp phá, giết hại dân lành. Đứng đầu đám giặc cỏ là tên tướng Phạ Chẩu Tin Toòng, có thuyết gọi là Phạ Chẩu Tín Toòng (ông tướng nhà trời).\nKhoảng năm 1740, giặc Phẻ chiếm được Mường Thanh và đóng quân trong thành Tam Vạn, rồi cướp phá khắp nơi đến tận Thuận Châu (Sơn La).\nCó hai thủ lĩnh người Thái là Lò Văn Ngải, Lò Văn Khanh đứng lên tập hợp, lãnh đạo dân Mường Thanh chống giặc. Song lực còn yếu, nghĩa quân chịu nhiều tổn thất, phải rút lên vùng núi cao bảo toàn lực lượng.\nNăm 1751, nghe tin có vị tướng miền xuôi - tức thủ lĩnh nghĩ

In [90]:
categories = page.categories
for title in sorted(categories.keys()):
    print("%s: %s" % (title, categories[title]))

Thể loại:Bản mẫu webarchive dùng liên kết wayback: Thể loại:Bản mẫu webarchive dùng liên kết wayback (lang: vi, variant: None, id: ??, ns: 14)
Thể loại:Di tích quốc gia Việt Nam: Thể loại:Di tích quốc gia Việt Nam (lang: vi, variant: None, id: ??, ns: 14)
Thể loại:Di tích tại Điện Biên: Thể loại:Di tích tại Điện Biên (lang: vi, variant: None, id: ??, ns: 14)
Thể loại:Thành cổ Việt Nam: Thể loại:Thành cổ Việt Nam (lang: vi, variant: None, id: ??, ns: 14)
Thể loại:Tọa độ không có sẵn trên Wikidata: Thể loại:Tọa độ không có sẵn trên Wikidata (lang: vi, variant: None, id: ??, ns: 14)
Thể loại:Điện Biên: Thể loại:Điện Biên (lang: vi, variant: None, id: ??, ns: 14)


In [ ]:
page2 = wiki_wiki.page('Thành Bát Quái')
exclude_titles_lower = ["xem thêm", "chú thích", "liên kết ngoài", "tham khảo", "đọc thêm", "gallery", "hình ảnh"]
meaningful_text_parts = []
for section in page2.sections:
    if section.title.lower().strip() not in exclude_titles_lower:
        sec_text = section.text
        meaningful_text_parts.append(sec_text)
        print(f"Section: {section.title}\n{sec_text}\n")

full_meaningful_text = "\n\n".join(meaningful_text_parts) 

Section: Lịch sử
Tháng 8 năm Mậu Thân (7 tháng 9 năm 1788), lợi dụng khi quân Tây Sơn đang bận tái lập trật tự Bắc Hà và đánh quân Thanh, Nguyễn Ánh đánh chiếm được Sài Gòn và biến nơi đây thành cơ sở chống lại Tây Sơn.
Hai năm sau, 1790, Nguyễn Ánh chọn đất Sài Gòn làm kinh đô, tên là Gia Định kinh; rồi ông nhờ hai người Pháp là Olivier de Puymanel (Việt danh là "Ông Tín") và Théodore Lebrun là 2 sĩ quan công binh người Pháp đi theo giám mục Bá Đa Lộc vẽ họa đồ thành theo kiểu châu Âu kết hợp với bản vẽ quy hoạch của Trần Văn Học. Thành được xây dưới sự chỉ huy của Tôn Thất Hội với gần 30.000 dân phu được huy động, với tường thành cao mười lăm thước mộc, tính ra lối bốn thước tây lẻ tám tấc (khoảng 4 m 8), toàn bằng đá ong Biên Hòa kiểu "lục lăng", nhằm củng cố chân đứng của mình trên đất Gia Định.
Tháng 7 năm Nhâm Tý (1792), vua Quang Trung Nguyễn Huệ của nhà Tây Sơn băng hà, triều Tây Sơn lục đục nội bộ và rơi vào khủng hoảng, Nguyễn Ánh nhanh chóng tổ chức phản công và ông đánh bại

In [91]:

def print_categorymembers(categorymembers, level=0, max_level=1):
    for c in categorymembers.values():
        print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)

cat = wiki_wiki.page("Category:Thành cổ Việt Nam")
print("Category members: Category:Thành cổ Việt Nam")
print_categorymembers(cat.categorymembers)


Category members: Category:Thành cổ Việt Nam
*: Thành Bản Phủ (Điện Biên) (ns: 0)
*: Thành Bình Định (ns: 0)
*: Thành Bản Phủ (Cao Bằng) (ns: 0)
*: Thành Bát Quái (ns: 0)
*: Thành cổ Biên Hòa (ns: 0)
*: Bình Lỗ (ns: 0)
*: Đại đồn Chí Hòa (ns: 0)
*: Thành Cổ Loa (ns: 0)
*: Cổ Lũy (ns: 0)
*: Cửa Bắc (Thành Hà Nội) (ns: 0)
*: Thành cổ Diên Khánh (ns: 0)
*: Thành Đa Bang (ns: 0)
*: Đại La (ns: 0)
*: Đạo thành Đại Nài (ns: 0)
*: Đồ Bàn (ns: 0)
*: Khu di tích Đỗ Động Giang (ns: 0)
*: Thành Đồng Hới (ns: 0)
*: Thành Gia Định (ns: 0)
*: Thành Hoàng Đế (ns: 0)
*: Hạc thành (ns: 0)
*: Thành Hải Dương (ns: 0)
*: Quần thể di tích Cố đô Hoa Lư (ns: 0)
*: Hoàng thành Huế (ns: 0)
*: Hồi Hồ (ns: 0)
*: Kinh thành Huế (ns: 0)
*: Quần thể di tích Cố đô Huế (ns: 0)
*: Thành Hưng Hóa (ns: 0)
*: Kandapurpura (ns: 0)
*: Khu Túc (ns: 0)
*: Kottinagar (ns: 0)
*: La Thành Tây Đô (ns: 0)
*: Lũy Thầy (ns: 0)
*: Thành cổ Lưỡng Kỳ (ns: 0)
*: Thành cổ Núi Thành (ns: 0)
*: Thành Phụng (ns: 0)
*: Phượng Hoàng trung đô

In [ ]:
def get_wiki_lang_and_title_from_url(url):
    try:
        parsed_url = urllib.parse.urlparse(url)
        hostname = parsed_url.hostname # 'vi'
        path_parts = parsed_url.path.strip('/').split('/') # ['wiki', 'title']

        lang_match = re.match(r"([a-z]{2,3}(?:-[a-z]+)?)", hostname)
        if not lang_match:
            print(f"Error: Could not extract language from hostname: {hostname}")
            return None, None
        lang = lang_match.group(1)

        if len(path_parts) >= 2 and path_parts[0].lower() == 'wiki':
            page_title_encoded = path_parts[1]
            page_title = urllib.parse.unquote(page_title_encoded).replace('_', ' ')
            return lang, page_title
        else:
            print(f"Error: URL path does not match expected '/wiki/Page_Title' structure: {parsed_url.path}")
            return None, None

    except Exception as e:
        print(f"Error parsing {url}': {e}")
        return None, None
    
get_wiki_lang_and_title_from_url('https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_Di_t%C3%ADch_qu%E1%BB%91c_gia_Vi%E1%BB%87t_Nam')

('vi', 'Danh sách Di tích quốc gia Việt Nam')

In [ ]:
def process_wikipedia_page_and_links(seed_url, max_links_to_process=5, text_chars_limit=200):
    """
    Fetches a Wikipedia page, then iterates through its internal links,
    printing the title and summary/text of each linked page.
    """
    lang, seed_page_title = get_wiki_lang_and_title_from_url(seed_url)

    # if not lang or not seed_page_title:
    #     print("Could not proceed due to URL parsing issues.")
    #     return

    # print(f"Processing seed page: '{seed_page_title}' on {lang}.wikipedia.org")

    wiki_instance = wikipediaapi.Wikipedia(language=lang, user_agent=ua.random)
    seed_page_object = wiki_instance.page(seed_page_title)

    # if not seed_page_object.exists():
    #     print(f"Seed page '{seed_page_title}' does not exist on {lang} Wikipedia.")
    #     return

    print(f"\n--- Links from '{seed_page_object.title}' ---")
    links = seed_page_object.links # dict: {'Linked Page Title': WikipediaPage object}

    if not links:
        print("No internal links found on the seed page.")
        return
    for key in links.keys():
        print(key)
    processed_count = 0
    for linked_page_title, linked_page_object in links.items():
        if processed_count >= max_links_to_process:
            print(f"\nReached limit of {max_links_to_process} linked pages to process.")
            break

        print(f"\n  🔗 Linked Page Title: {linked_page_title}")

        # The linked_page_object is already a WikipediaPage object.
        # Accessing its properties like .summary or .text might trigger an API call
        # by the library if the data hasn't been fetched yet.
        if linked_page_object.exists():
            # Use summary for brevity, or .text for more content
            summary = linked_page_object.summary
            print(f"     Summary (first {text_chars_limit} chars): {summary[:text_chars_limit]}...")
        else:
            print(f"     (Linked page '{linked_page_title}' does not exist or is a special page)")

        processed_count += 1
        time.sleep(1) # Be polite to the API, especially if processing many links

    print(f"\nFinished processing links from '{seed_page_object.title}'.")
    return links


print("-" * 40)

english_python_url = 'https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_Di_t%C3%ADch_qu%E1%BB%91c_gia_Vi%E1%BB%87t_Nam'
links = process_wikipedia_page_and_links(english_python_url, max_links_to_process=3)

----------------------------------------

--- Links from 'Danh sách Di tích quốc gia Việt Nam' ---
ATK2
ATK Chợ Đồn
ATK Định Hóa
Am Chúa
An Biên (phường)
An Bài, Quỳnh Phụ
An Cựu
An Dương
An Dục, Quỳnh Phụ
An Khê
An Khê, Quỳnh Phụ
An Lão
An Lễ, Quỳnh Phụ
An Lộc, Bình Long
An Nhơn
An Ninh, Quỳnh Phụ
An Ninh, Tiền Hải
An Phú
An Thái, Quỳnh Phụ
An Thạnh Đông, Cù Lao Dung
An Tràng, Quỳnh Phụ
An Tân
An Tây
An Tường, Vĩnh Tường
An toàn khu Định Hóa
Ao Bà Om
Ayun Hạ
Ba Bể
Ba Chẽ
Ba Tri
Ba Tơ
Ba Vì
Ba Đình
Ba Đồn
Bia Lê Lợi
Bia Ma Nhai
Bia Quế Lâm Ngự Chế
Bia mộ Nguyễn Trường Tộ
Bia tiến sĩ Văn Miếu Thăng Long
Biên Hòa
Buôn Ma Thuột
Buôn Đôn
Bà Rịa
Bà Triệu (phường)
Bàu Trắng
Bách Thuận, Vũ Thư
Bát Xát
Bãi cọc Yên Giang
Bãi đá cổ Nấm Dẩn
Bãi đá cổ Sa Pa
Bãi đá khắc cổ Khe Hổ
Bình Dương, Vĩnh Tường
Bình Gia
Bình Lục
Bình Thanh
Bình Thủy
Bình Đà
Bình Đại
Bình Định Bắc
Bích Câu đạo quán
Bù Gia Mập
Bùi Cầm Hổ
Bùi Hữu Nghĩa
Bùi Sỹ Tiêm
Bùi Thị Xuân
Bùi Viện
Bạc Liêu (thành phố)
Bạch Dinh
Bạch Liêu


In [30]:
links['Nhà số 7 phố Bến Ngự'].summary

''

In [32]:
wiki_wiki.page('Nhà số 7 phố Bến Ngự')

Nhà số 7 phố Bến Ngự (lang: en, variant: None, id: ??, ns: 0)